In [ ]:
!pip install accelerate

In [4]:
import transformers
import accelerate

# Load the pre-trained model
model = transformers.AutoModel.from_pretrained("facebook/bart-base")

# Create an Accelerate Accelerator
accelerator = accelerate.Accelerator()

# Move the model to the GPU
model.to(accelerator.device)

BartModel(
  (shared): Embedding(50265, 768, padding_idx=1)
  (encoder): BartEncoder(
    (embed_tokens): Embedding(50265, 768, padding_idx=1)
    (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
    (layers): ModuleList(
      (0-5): 6 x BartEncoderLayer(
        (self_attn): BartAttention(
          (k_proj): Linear(in_features=768, out_features=768, bias=True)
          (v_proj): Linear(in_features=768, out_features=768, bias=True)
          (q_proj): Linear(in_features=768, out_features=768, bias=True)
          (out_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (activation_fn): GELUActivation()
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      )
    )
    (layernorm_embedding): La

In [6]:
import csv

def load_dataset(csv_file):
    """Loads a code migration dataset from a CSV file.

    Args:
        csv_file: The path to the CSV file.

    Returns:
        A list of tuples, where each tuple contains the input and output code snippets.
    """

    dataset = []
    with open(csv_file, "r") as f:
        reader = csv.reader(f)

        for row in reader:
            input_code = row[0]
            output_code = row[1]

            dataset.append((input_code, output_code))

    return dataset

# Load the modified dataset
dataset = load_dataset("Data\code_to_code_geekforgeek.csv")

In [16]:
from transformers import AutoTokenizer
from torch.utils.data import DataLoader

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")

def create_train_dataloader(dataset, batch_size):
    """Creates a training dataloader for a code migration dataset.

    Args:
        dataset: A list of tuples, where each tuple contains the input and output code snippets.
        batch_size: The batch size.

    Returns:
        A DataLoader object.
    """

    train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    return train_dataloader

# Create the training dataloader
train_dataloader = create_train_dataloader(dataset, batch_size=16)


In [10]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=1e-5)

c:\Users\divya\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [12]:
from transformers import get_linear_schedule_with_warmup

num_training_steps = len(train_dataloader) * 2000

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=1000, num_training_steps=num_training_steps)


In [17]:
# Start training
for epoch in range(2000):
    for batch in train_dataloader:
        # Convert the list of code snippets to tensors
        batch = tokenizer(batch, return_tensors="pt")
        # Move the batch to the GPU
        batch = batch.to(accelerator.device)
        print("To forward pass")
        # Forward pass
        outputs = model(**batch)

        # Loss
        print("To loss")
        loss = outputs.loss

        # Backward pass
        print("To backward pass")
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Update the learning rate
        print("Updating learning rate")
        scheduler.step()

# Save the fine-tuned model
model.save_pretrained("fine-tuned-bart-base-code-migration")

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]